## [부산대학교 역사 데이터 수집하기](https://www.pusan.ac.kr/kor/CMS/HistoryMgr/list.do?mCode=MN164)
### 2022.01.27

In [1]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [2]:
# 요청함수
def getSource(site) :
    # 헤더 정보 셋팅
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()

In [7]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, fileName) :
    
    # 데이터 있는 전체를 가져온다.
    a1 = source.select_one('#tab-items')
    # print(a1)
    
    data_dict = {
        '년도' : [],
        '월' : [],
        '내용' : []
    }
    
    year = 2020
    # 항목의 수 만큼 반복한다.
    for _ in range(9) :
        # 하위 항목을 가져온다.
        a2 = a1.select_one(f'#HIS_{year}')
        # print(a2)
        
        # 년도 하나를 가져온다.
        a3 = a2.select('dl')
        
        for a4 in a3 :
            # print(a4)
            
            # 년도를 추출한다.
            a5 = a4.select_one('dt > span')
            data1 = a5.text.strip()
            # '년'을 제외한다.
            data1 = data1[:-1].strip()
            # print(data1)
            
            # 월별 데이터 전체를 가져온다.
            a6 = a4.select_one('dd')
            # print(data1)
            # print(a6)
            # 월을 가져온다.
            a7 = a6.select('h3')
            
            # 연역 내용 전체를 가져온다.
            a8 = a6.select('ul')
            
            for idx in range(len(a7)) :
                # 월 하나를 가져온다.
                a9 = a7[idx]
                data2 = a9.text.strip()
                data2 = data2[:-1].strip()
                # print(data2)
                
                # 연혁 한덩어리를 가져온다.
                a10 = a8[idx]
                # 세부 연혁들을 가져온다.
                a11 = a10.select('li')
                
                for a12 in a11 :
                    data3 = a12.text.strip()
                    # print(data1, data2, data3)
                    # print('-------------------------')
                    
                    # 저장할 데이터를 담는다.
                    data_dict['년도'].append(data1)
                    data_dict['월'].append(data2)
                    data_dict['내용'].append(data3)
    
        year = year - 10
        
    # 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    # 저장한다.
    # 파일이 없다면 : 첫 번째 저장
    if os.path.exists(fileName) == False :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False)
    # 파일이 있다면..
    else :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False, mode='a', header=None)

In [4]:
# 다음 페이지 주소를 가져오는 함수
def getNextPage(source) :
    pass

In [5]:
# 페이지의 주소
site = 'https://www.pusan.ac.kr/kor/CMS/HistoryMgr/list.do?mCode=MN164'
nextPage = ''
fileName = 'data2.csv'

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)
    
# 서버에 접속해서 응답 결과를 가져온다.
a1 = getSource(f'{site}{nextPage}')
source = bs4.BeautifulSoup(a1)

# 현재 페이지의 데이터를 수집한다.
getData(source, fileName)

print('수집 완료')

수집 완료


In [6]:
pd.read_csv(fileName)

,년도,월,내용
0,2020,3,정보의생명공학대학 신설
1,2020,5,제21대 총장 차정인 박사 취임
2,2019,3,통일한국연구원 개원
3,2019,5,주한독일문화원 부산분원 개원
4,2019,10,부마민주항쟁 40주년 기념식(국가기념일 지정)
...,...,...,...
353,1946,5,"경상남도 학무과장 윤인구, 미고문관 에디 중위가 종합대학교 설립안(교명 : 교남대학..."
354,1946,5,"교명을 부산대학으로 하고, 인문학부와 수산학부로 구성된 국립종합대학교 설립 확정"
355,1946,7,"예과 1학년 모집, 입학시험 실시, 인문과대학, 수산과대학 각각 150명 선발"
356,1946,9,"개교식, 제1회 입학식(예과 1학년) 및 총장 베커 박사 취임식 거행"
